# Reinforcement Learning, 2024-01
## Tarea 4 - Algoritmos de aprendizaje por refuerzo con aproximación de funciones

> Daniel Villar González, 201923374.  
> Daniel Alvarez, 201911320.

## **Descripción de tarea**

Considere el problema Cart Pole implementado en el entorno de Gymnasium descrito. El objetivo de este taller es comparar algoritmos de RL tabulares con sus contrapartes que utilizan aproximación de funciones.

### ***Requerimientos***
1. La recompensa vista en un estado terminal es cero.
2. El siguiente estado visto por un agente en un estado terminal es igual al mismo, cumpliendo con las dinámicas del MDP.
3. Para este caso, como existe aprendizaje, no se conocen de primera mano las probabilidades de transición, por lo que el agente debe aprender.
4. Existen varios episodios donde se actualiza la matriz Q, con el fin de conocer los mayores valores del par estado-acción.

## **Librerias**

In [2]:
import numpy as np
import random
import gym

### ***Pruebas de entorno***

- A continuación se muestra los test realizados para la obtención de las respuestas del agente ante un ambiente determinado para la modalidad de "CartPole" 

In [2]:
def Random_games():
    env = gym.make("CartPole-v1")
    # Each of this episode is its own game.
    for episode in range(10):
        env.reset()
        total_reward = 0
        # this is each frame, up to 500...but we wont make it that far with random.
        for t in range(500):
            env.render()
            
            # This will just create a sample action in any environment.
            # In this environment, the action can be 0 or 1, which is left or right
            action = env.action_space.sample()

            # this executes the environment with an action, 
            # and returns the observation of the environment, 
            # the reward, if the env is over, and other info.
            #print(env.step(action))
            next_state, reward, done, info, _ = env.step(action)
            total_reward += reward
            # lets print everything in one line:
             # Print information
            print("Step:", t, " Action:", action, " Reward:", reward, "NextState:", next_state, " Done:", done, "Info:", info)
            
            if done:
                print("Episode", episode, "terminated after", t+1, "timesteps with total reward", total_reward)
                if total_reward>=500:
                    print("El episodio se ha completado de manera exitosa.")
                else:
                    print("El episodio no se completó exitosamente.")
                break  # Exit the loop if the episode is done
                
Random_games()

C:\Users\danie\AppData\Roaming\Python\Python311\site-packages\gym\envs\classic_control\cartpole.py:211: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("CartPole-v1", render_mode="rgb_array")
  gym.logger.warn(
C:\Users\danie\AppData\Roaming\Python\Python311\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Step: 0  Action: 1  Reward: 1.0 NextState: [-0.03191042  0.228863    0.04778132 -0.30931753]  Done: False Info: False
Step: 1  Action: 0  Reward: 1.0 NextState: [-0.02733316  0.03309398  0.04159497 -0.00195676]  Done: False Info: False
Step: 2  Action: 1  Reward: 1.0 NextState: [-0.02667128  0.22759548  0.04155584 -0.28123125]  Done: False Info: False
Step: 3  Action: 0  Reward: 1.0 NextState: [-0.02211937  0.03190615  0.03593121  0.0242632 ]  Done: False Info: False
Step: 4  Action: 0  Reward: 1.0 NextState: [-0.02148125 -0.16371216  0.03641647  0.3280628 ]  Done: False Info: False
Step: 5  Action: 1  Reward: 1.0 NextState: [-0.02475549  0.03087293  0.04297773  0.04708274]  Done: False Info: False
Step: 6  Action: 0  Reward: 1.0 NextState: [-0.02413804 -0.1648381   0.04391939  0.35300976]  Done: False Info: False
Step: 7  Action: 1  Reward: 1.0 NextState: [-0.0274348   0.0296327   0.05097958  0.07449301]  Done: False Info: False
Step: 8  Action: 1  Reward: 1.0 NextState: [-0.02684214 

[-0.06963672 -0.82801914  0.06049993  1.2680922 ]  Done: False Info: False
Step: 12  Action: 0  Reward: 1.0 NextState: [-0.0861971  -1.0238594   0.08586177  1.5790912 ]  Done: False Info: False
Step: 13  Action: 1  Reward: 1.0 NextState: [-0.10667429 -0.82985854  0.1174436   1.3143743 ]  Done: False Info: False
Step: 14  Action: 1  Reward: 1.0 NextState: [-0.12327146 -0.6364023   0.14373109  1.0606381 ]  Done: False Info: False
Step: 15  Action: 0  Reward: 1.0 NextState: [-0.1359995  -0.8331048   0.16494384  1.394761  ]  Done: False Info: False
Step: 16  Action: 0  Reward: 1.0 NextState: [-0.1526616  -1.0298499   0.19283906  1.734147  ]  Done: False Info: False
Step: 17  Action: 1  Reward: 1.0 NextState: [-0.1732586 -0.8373804  0.227522   1.5071381]  Done: True Info: False
Episode 5 terminated after 18 timesteps with total reward 18.0
El episodio no se completó exitosamente.
Step: 0  Action: 0  Reward: 1.0 NextState: [ 0.0282857  -0.21653746 -0.00543063  0.28746888]  Done: False Info: 

### ***Punto 1***

**Resuelva el problema con SARSA y con Q-Learning tabulares (su propia implementación). Para esto debe discretizarse el vector de estados. Utilice dos discretizaciones:**
**a) Una discretización cruda, de manera que el espacio de estados tenga ∼ 300 estados.**
**b) Una discretización más fina, de manera que el espacio de estados tenga ∼ 3000 estados.**

En este estudio, se utilizó el algoritmo de SARSA y el algoritmo de Q-learning, ambos implementados de forma tabular para la solución del problema conocido como ***CartPole***. Estos métodos son fundamentales para el aprendizaje por refuerzo y se basan en la actualización de la matriz de función de valor acción-estado, \(Q(S, A)\), donde \(S\) representa el estado y \(A\) la acción.

### Q-learning
El algoritmo de Q-learning es un método de aprendizaje por refuerzo sin modelo que busca la política óptima de manera 'off-policy', es decir, aprendiendo de las acciones que no necesariamente son parte de la política actual. La actualización de la matriz Q en Q-learning se realiza según la siguiente regla:
$$
Q(S, A) \leftarrow Q(S, A) + \alpha \left[R + \gamma \max_{a} Q(S', a) - Q(S, A)\right]
$$
Donde:
- \(S'\) es el estado siguiente al tomar la acción \(A\) en el estado \(S\).
- \(R\) es la recompensa recibida tras tomar la acción \(A\) en el estado \(S\).
- $(\alpha)$ es la tasa de aprendizaje, que determina cuánto influyen las nuevas informaciones en la actualización.
- $(\gamma)$ es el factor de descuento, que modula la importancia de las recompensas futuras.

### SARSA
Por otro lado, SARSA (State-Action-Reward-State-Action) es un algoritmo 'on-policy', que evalúa y mejora la política que está siendo realmente seguida. La actualización de la matriz Q en SARSA sigue esta regla:
$$
Q(S, A) \leftarrow Q(S, A) + \alpha \left[R + \gamma Q(S', A') - Q(S, A)\right]
$$
Donde \(A'\) es la acción que se toma en el estado siguiente \(S'\) según la política actual.



----

### ***Caso SARSA***  
- **Discretización 300 estados.**  
Para realizar la discretización utilizamos la libreria "gym", la cual nos aporta alrededor de 4 valores importantes en el estudio del cartpole: su posición, velocidad, ángulo del palo y por último, velocidad angular de este. Por lo que cada estado se define como una combinación de estos 4 valores críticos en la dinámica del modelo, teniendo esto en cuenta tomaremos las divisiones del estado lo más cerca posible a 300, resolviendo:
$$300 = BINS^{4}$$
Obteniendo el valor de división
$$BINS \approx 4.1617$$
Tomando su aproximación superior tomaremos 5. Obteniendo el siguiente modelo:

In [4]:
class SarsaAgent:
    def __init__(self, num_states, num_actions, discount_factor=0.99, alpha=0.1, epsilon=0.1):
        self.q_table = np.zeros((num_states, num_actions))
        self.discount_factor = discount_factor
        self.alpha = alpha
        self.epsilon = epsilon
        self.epsilon_init = epsilon

    def choose_action(self, state):
        if np.random.rand() < self.epsilon:
            return np.random.choice(len(self.q_table[state]))
        else:
            return np.argmax(self.q_table[state])

    def update_q_table(self, state, action, reward, next_state, next_action):
        predict = self.q_table[state, action]
        target = reward + self.discount_factor * self.q_table[next_state, next_action]
        self.q_table[state, action] += self.alpha * (target - predict)

    def reset_epsilon(self):
        self.epsilon = self.epsilon_init

# Función para discretizar el espacio de estados

def discretize_state(state, num_bins):
    bins = [
        np.linspace(-4.8, 4.8, num_bins),  # Posición del carro
        np.linspace(-3, 3, num_bins),      # Velocidad del carro
        np.linspace(-0.418, 0.418, num_bins),  # Ángulo del palo
        np.linspace(-3, 3, num_bins)       # Velocidad angular del palo
    ]
    discretized_indices = [np.digitize(state[i], bins[i]) - 1 for i in range(len(state))]
    single_index = sum(discretized_indices[i] * (num_bins ** i) for i in range(len(discretized_indices)))
    return single_index


# Función para entrenar el agente SARSA
def train_sarsa(env, agent, num_bins_t, num_episodes):
    num_bins = num_bins_t  # Se ajusta este valor según la granularidad deseada
    for episode in range(num_episodes):
        initial_state, _ = env.reset()  # Accede al estado inicial
        state = discretize_state(initial_state, num_bins)
        action = agent.choose_action(state)
        terminal = False
        print("___________________")
        print("episodio: " + str(episode))
        while not terminal:
            next_state_env, reward, terminal, truncated, _ = env.step(action)
            next_state = discretize_state(next_state_env, num_bins)
            next_action = agent.choose_action(next_state)
            agent.update_q_table(state, action, reward, next_state, next_action)
            state = next_state
            action = next_action
        agent.reset_epsilon()

# Función para obtener la política del agente
def get_policy(q_table):
    return np.argmax(q_table, axis=1)

# Crear el entorno CartPole-v1

env = gym.make("CartPole-v1")

# Definir el agente SARSA
num_bins = 5
num_state_variables = 4
# El numero de estados seria los bins elevados a la cantidad de variables por estado

num_states = num_bins ** num_state_variables
num_actions = env.action_space.n
sarsa_agent = SarsaAgent(num_states, num_actions)

# Entrenar el agente SARSA
train_sarsa(env, sarsa_agent, num_bins, num_episodes=1000)

# Obtener la política del agente
policy = get_policy(sarsa_agent.q_table)
print("Política aprendida:", policy)


___________________
episodio: 0
___________________
episodio: 1
___________________
episodio: 2
___________________
episodio: 3
___________________
episodio: 4
___________________
episodio: 5
___________________
episodio: 6
___________________
episodio: 7
___________________
episodio: 8
___________________
episodio: 9
___________________
episodio: 10
___________________
episodio: 11
___________________
episodio: 12
___________________
episodio: 13
___________________
episodio: 14
___________________
episodio: 15
___________________
episodio: 16
___________________
episodio: 17
___________________
episodio: 18
___________________
episodio: 19
___________________
episodio: 20
___________________
episodio: 21
___________________
episodio: 22
___________________
episodio: 23
___________________
episodio: 24
___________________
episodio: 25
___________________
episodio: 26
___________________
episodio: 27
___________________
episodio: 28
___________________
episodio: 29
___________________


c:\Users\danie\AppData\Local\Programs\Python\Python38\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


___________________
episodio: 96
___________________
episodio: 97
___________________
episodio: 98
___________________
episodio: 99
___________________
episodio: 100
___________________
episodio: 101
___________________
episodio: 102
___________________
episodio: 103
___________________
episodio: 104
___________________
episodio: 105
___________________
episodio: 106
___________________
episodio: 107
___________________
episodio: 108
___________________
episodio: 109
___________________
episodio: 110
___________________
episodio: 111
___________________
episodio: 112
___________________
episodio: 113
___________________
episodio: 114
___________________
episodio: 115
___________________
episodio: 116
___________________
episodio: 117
___________________
episodio: 118
___________________
episodio: 119
___________________
episodio: 120
___________________
episodio: 121
___________________
episodio: 122
___________________
episodio: 123
___________________
episodio: 124
__________________

In [10]:
def train_and_evaluate(env_name, agent, num_bins, num_episodes, num_experiments):
    # Lista para almacenar las recompensas acumuladas de cada episodio en cada experimento
    all_rewards = []
    average_rewards = [0]*num_experiments
    
    # Entrenamiento y evaluación en múltiples experimentos
    for experiment in range(num_experiments):
        env = gym.make(env_name)
        rewards = 0
        
        for episode in range(num_episodes):
            initial_state = env.reset()
            state = discretize_state(list(initial_state[0]), num_bins)
            action = agent.choose_action(state)
            total_reward = 0
            
            terminal = False
            while not terminal:
                next_state_env, reward, terminal, truncated, _ = env.step(action)
                next_state = discretize_state(next_state_env, num_bins)
                next_action = agent.choose_action(next_state)
                agent.update_q_table(state, action, reward, next_state, next_action)
                state = next_state
                action = next_action
                total_reward += reward
            rewards += total_reward
            agent.reset_epsilon()
        
        all_rewards.append(rewards)
        average_rewards[experiment] = np.mean(all_rewards, axis=0)
        
    # Evaluar la política resultante
    final_policies = []
    for _ in range(num_experiments):
        final_policies.append(get_policy(agent.q_table))
    
    return average_rewards, final_policies

# Configuración de parámetros
env_name = "CartPole-v1"
num_bins = 5
num_episodes = 100
num_experiments = 100

# Crear el entorno CartPole-v1
env = gym.make(env_name)

# Definir el agente SARSA
num_state_variables = 4
num_states = num_bins ** num_state_variables
num_actions = env.action_space.n
sarsa_agent = SarsaAgent(num_states, num_actions)

# Entrenar y evaluar el agente SARSA
average_rewards, final_policies = train_and_evaluate(env_name, sarsa_agent, num_bins, num_episodes, num_experiments)

# Mostrar promedios de recompensas acumuladas
print("Promedio de recompensas acumuladas en cada episodio:")
print(average_rewards)

# Mostrar políticas finales
print("Políticas finales:")
for i, policy in enumerate(final_policies):
    print("Experimento", i+1, ":", policy)
politica_final = final_policies[-1]

KeyboardInterrupt: 

In [5]:
len(politica_final)

625

### **Prueba de la política**

In [6]:
import gym
import numpy as np
import time
env = gym.make('CartPole-v1',render_mode='human')

(state,_)=env.reset()
env.render()
env.step(0)
env.observation_space
env.observation_space.high
env.observation_space.low
env.action_space
env.spec
env.spec.max_episode_steps
env.spec.reward_threshold
# simulate the environment
episodeNumber=1
timeSteps=100

def discretize_state(state, num_bins):
    bins = [
        np.linspace(-4.8, 4.8, num_bins),  # Posición del carro
        np.linspace(-3, 3, num_bins),      # Velocidad del carro
        np.linspace(-0.418, 0.418, num_bins),  # Ángulo del palo
        np.linspace(-3, 3, num_bins)       # Velocidad angular del palo
    ]
    discretized_indices = [np.digitize(state[i], bins[i]) - 1 for i in range(len(state))]
    single_index = sum(discretized_indices[i] * (num_bins ** i) for i in range(len(discretized_indices)))
    return single_index

def choose_action_with_policy(state, policy):
    print(state)
    state_index = discretize_state(state, 5)
    return policy[state_index]

policy = politica_final


for episodeIndex in range(episodeNumber):
    state,_=env.reset()
    env.render()
    appendedObservations=[]
    for timeIndex in range(timeSteps):
        action = choose_action_with_policy(state, policy)
        next_state_env, reward, terminal, truncated, _ = env.step(action)
        state = next_state_env   
        appendedObservations.append(next_state_env)
        time.sleep(0.1)
        if (terminal):
            time.sleep(1)
            break
env.close() 

[ 0.01683148 -0.01405446 -0.02650084  0.00333339]
[ 0.01655039 -0.20878652 -0.02643417  0.2875386 ]
[ 0.01237466 -0.40352172 -0.0206834   0.5717686 ]
[ 0.00430423 -0.5983476  -0.00924803  0.85786444]
[-0.00766272 -0.79334235  0.00790926  1.1476252 ]
[-0.02352957 -0.5983246   0.03086177  0.85743296]
[-0.03549606 -0.4036364   0.04801043  0.57461166]
[-0.04356879 -0.20921922  0.05950266  0.29743156]
[-0.04775317 -0.01499373  0.06545129  0.0240924 ]
[-0.04805305  0.17913151  0.06593314 -0.24724302]
[-0.04447042 -0.0168671   0.06098828  0.06548639]
[-0.04480776  0.17732982  0.06229801 -0.20734797]
[-0.04126116 -0.01862508  0.05815105  0.10431794]
[-0.04163367  0.17561738  0.0602374  -0.1694665 ]
[-0.03812132 -0.02031272  0.05684808  0.14159472]
[-0.03852757  0.17395093  0.05967997 -0.1326257 ]
[-0.03504856 -0.02197287  0.05702746  0.17827229]
[-0.03548801  0.17228858  0.0605929  -0.09588922]
[-0.03204224 -0.02364712  0.05867512  0.21527837]
[-0.03251518  0.17058906  0.06298068 -0.0583339 ]


Realizando el mismo procedimiento en este caso para 3000 estados, solveremos la siguiente expresión:
$$3000 = BINS^{4}$$
Obteniendo
$$BINS \approx 7.4$$
Aproximando este valor a 8 unidades de subdivisiones de los datos continuos.

In [7]:
# Crear el entorno CartPole-v1

env = gym.make("CartPole-v1")

# Definir el agente SARSA
num_bins = 8
num_state_variables = 4
# El numero de estados seria los bins elevados a la cantidad de variables por estado

num_states = num_bins ** num_state_variables
num_actions = env.action_space.n
sarsa_agent = SarsaAgent(num_states, num_actions)

# Entrenar el agente SARSA
train_sarsa(env, sarsa_agent, num_bins, num_episodes=1000)

# Obtener la política del agente
policy2 = get_policy(sarsa_agent.q_table)
print("Política aprendida:", policy2)

___________________
episodio: 0
___________________
episodio: 1
___________________
episodio: 2
___________________
episodio: 3
___________________
episodio: 4
___________________
episodio: 5
___________________
episodio: 6
___________________
episodio: 7
___________________
episodio: 8
___________________
episodio: 9
___________________
episodio: 10
___________________
episodio: 11
___________________
episodio: 12
___________________
episodio: 13
___________________
episodio: 14
___________________
episodio: 15
___________________
episodio: 16
___________________
episodio: 17
___________________
episodio: 18
___________________
episodio: 19
___________________
episodio: 20
___________________
episodio: 21
___________________
episodio: 22
___________________
episodio: 23
___________________
episodio: 24
___________________
episodio: 25
___________________
episodio: 26
___________________
episodio: 27
___________________
episodio: 28
___________________
episodio: 29
___________________


In [8]:
for i in range(len(policy2)):
    print(policy2[i])

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


### ***Caso Q-Learning***  
Para este caso se estableció las mismas divisiones usadas para el caso de SARSA, adaptando de igual forma la forma de entrenamiento de este para poder discretizarlo en valores que se combinen con cada uno de los estados:

In [9]:
class QLearningAgent:
    def __init__(self, num_states, discount_factor=0.99, alpha=0.5, epsilon=0.1):
        self.q_table = np.zeros((num_states, num_actions))  # 2 acciones
        self.discount_factor = discount_factor
        self.alpha = alpha
        self.epsilon = epsilon

    def choose_action(self, state):
        if np.random.rand() < self.epsilon:
            return 0 if np.random.rand() < 0.5 else 1  # Acción aleatoria
        else:
            action_index = np.argmax(self.q_table[state])
            return 0 if action_index == 0 else 1  # Mejor acción conocida

    def update_q_table(self, state, action, reward, next_state):
        action_index = 0 if action == 0 else 1
        predict = self.q_table[state, action_index]
        target = reward + self.discount_factor * np.max(self.q_table[next_state])
        self.q_table[state, action_index] += self.alpha * (target - predict)

def train_q_learning(env, agent, num_bins_t, num_episodes):
    num_bins = num_bins_t  # Ajusta este valor según la granularidad deseada
    for episode in range(num_episodes):
        initial_state, _ = env.reset()  # Accede al estado inicial
        state = discretize_state(initial_state, num_bins)
        action = agent.choose_action(state)
        terminal = False
        print("___________________")
        print("episodio: " + str(episode))
        while not terminal:
            next_state_env, reward, terminal, truncated, _ = env.step(action)
            next_state = discretize_state(next_state_env, num_bins)
            next_action = agent.choose_action(next_state)
            agent.update_q_table(state, action, reward, next_state)
            state = next_state
            action = next_action

In [10]:
# Crear el entorno CartPole-v1

env = gym.make("CartPole-v1")

# Definir el agente SARSA
num_bins = 5
num_state_variables = 4
# El numero de estados seria los bins elevados a la cantidad de variables por estado

num_states = num_bins ** num_state_variables
num_actions = env.action_space.n
q_agent = QLearningAgent(num_states, num_actions)

# Entrenar el agente SARSA
train_q_learning(env, q_agent, num_bins, num_episodes=1000)

# Obtener la política del agente
policy3 = get_policy(sarsa_agent.q_table)
print("Política aprendida:", policy3)

___________________
episodio: 0
___________________
episodio: 1
___________________
episodio: 2
___________________
episodio: 3
___________________
episodio: 4
___________________
episodio: 5
___________________
episodio: 6
___________________
episodio: 7
___________________
episodio: 8
___________________
episodio: 9
___________________
episodio: 10
___________________
episodio: 11
___________________
episodio: 12
___________________
episodio: 13
___________________
episodio: 14
___________________
episodio: 15
___________________
episodio: 16
___________________
episodio: 17
___________________
episodio: 18
___________________
episodio: 19
___________________
episodio: 20
___________________
episodio: 21
___________________
episodio: 22
___________________
episodio: 23
___________________
episodio: 24
___________________
episodio: 25
___________________
episodio: 26
___________________
episodio: 27
___________________
episodio: 28


C:\Users\danie\AppData\Roaming\Python\Python311\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


___________________
episodio: 29
___________________
episodio: 30
___________________
episodio: 31
___________________
episodio: 32
___________________
episodio: 33
___________________
episodio: 34
___________________
episodio: 35
___________________
episodio: 36
___________________
episodio: 37
___________________
episodio: 38
___________________
episodio: 39
___________________
episodio: 40
___________________
episodio: 41
___________________
episodio: 42
___________________
episodio: 43
___________________
episodio: 44
___________________
episodio: 45
___________________
episodio: 46
___________________
episodio: 47
___________________
episodio: 48
___________________
episodio: 49
___________________
episodio: 50
___________________
episodio: 51
___________________
episodio: 52
___________________
episodio: 53
___________________
episodio: 54
___________________
episodio: 55
___________________
episodio: 56
___________________
episodio: 57
___________________
episodio: 58
__________

C:\Users\danie\AppData\Local\Temp\ipykernel_40168\3648342541.py:18: RuntimeWarning: overflow encountered in scalar multiply
  target = reward + self.discount_factor * np.max(self.q_table[next_state])
C:\Users\danie\AppData\Local\Temp\ipykernel_40168\3648342541.py:19: RuntimeWarning: invalid value encountered in scalar subtract
  self.q_table[state, action_index] += self.alpha * (target - predict)


___________________
episodio: 372
___________________
episodio: 373
___________________
episodio: 374
___________________
episodio: 375
___________________
episodio: 376
___________________
episodio: 377
___________________
episodio: 378
___________________
episodio: 379
___________________
episodio: 380
___________________
episodio: 381
___________________
episodio: 382
___________________
episodio: 383
___________________
episodio: 384
___________________
episodio: 385
___________________
episodio: 386
___________________
episodio: 387
___________________
episodio: 388
___________________
episodio: 389
___________________
episodio: 390
___________________
episodio: 391
___________________
episodio: 392
___________________
episodio: 393
___________________
episodio: 394
___________________
episodio: 395
___________________
episodio: 396
___________________
episodio: 397
___________________
episodio: 398
___________________
episodio: 399
___________________
episodio: 400
______________

In [11]:
for i in range(len(policy3)):
    print(policy3[i])

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [12]:
# Crear el entorno CartPole-v1

env = gym.make("CartPole-v1")

# Definir el agente SARSA
num_bins = 8
num_state_variables = 4
# El numero de estados seria los bins elevados a la cantidad de variables por estado

num_states = num_bins ** num_state_variables
num_actions = env.action_space.n
q_agent = QLearningAgent(num_states, num_actions)

# Entrenar el agente SARSA
train_q_learning(env, q_agent, num_bins, num_episodes=1000)

# Obtener la política del agente
policy4 = get_policy(sarsa_agent.q_table)
print("Política aprendida:", policy4)

___________________
episodio: 0
___________________
episodio: 1
___________________
episodio: 2
___________________
episodio: 3
___________________
episodio: 4
___________________
episodio: 5
___________________
episodio: 6
___________________
episodio: 7
___________________
episodio: 8
___________________
episodio: 9
___________________
episodio: 10
___________________
episodio: 11
___________________
episodio: 12
___________________
episodio: 13
___________________
episodio: 14
___________________
episodio: 15
___________________
episodio: 16
___________________
episodio: 17
___________________
episodio: 18
___________________
episodio: 19
___________________
episodio: 20
___________________
episodio: 21
___________________
episodio: 22
___________________
episodio: 23
___________________
episodio: 24
___________________
episodio: 25
___________________
episodio: 26
___________________
episodio: 27
___________________
episodio: 28
___________________
episodio: 29
___________________


C:\Users\danie\AppData\Local\Temp\ipykernel_40168\3648342541.py:18: RuntimeWarning: overflow encountered in scalar multiply
  target = reward + self.discount_factor * np.max(self.q_table[next_state])
C:\Users\danie\AppData\Local\Temp\ipykernel_40168\3648342541.py:19: RuntimeWarning: invalid value encountered in scalar subtract
  self.q_table[state, action_index] += self.alpha * (target - predict)


___________________
episodio: 338
___________________
episodio: 339
___________________
episodio: 340
___________________
episodio: 341
___________________
episodio: 342
___________________
episodio: 343
___________________
episodio: 344
___________________
episodio: 345
___________________
episodio: 346
___________________
episodio: 347
___________________
episodio: 348
___________________
episodio: 349
___________________
episodio: 350
___________________
episodio: 351
___________________
episodio: 352
___________________
episodio: 353
___________________
episodio: 354
___________________
episodio: 355
___________________
episodio: 356
___________________
episodio: 357
___________________
episodio: 358
___________________
episodio: 359
___________________
episodio: 360
___________________
episodio: 361
___________________
episodio: 362
___________________
episodio: 363
___________________
episodio: 364
___________________
episodio: 365
___________________
episodio: 366
______________

In [13]:
for i in range(len(policy4)):
    print(policy4[i])

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0



0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0

### ***Punto 2***
**Resuelva el problema con DeepSarsa y DQN, utilizando como aproximador una red neuronal con una capa escondida con 4 y 16 neuronas. Muestre promedios sobre múltiples experimentos de curvas de aprendizaje de entrenamiento. Evalúe las políticas resultantes en cada caso (en total 4 políticas) e incluya un link a un video del desempeño de cada política en el ambiente. Para este problema puede utilizar la librería keras-rl.**


In [12]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import random
import gym
import numpy as np
from collections import deque
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.optimizers import Adam, RMSprop
# Crear el entorno
env = gym.make('CartPole-v1')

In [11]:
def OurModel(input_shape, action_space):
    X_input = Input(input_shape)

    # Capa oculta con 4 neuronas
    X = Dense(4, activation="relu", kernel_initializer='he_uniform')(X_input)

    # Capa oculta con 16 neuronas
    X = Dense(16, activation="relu", kernel_initializer='he_uniform')(X)

    # Capa de salida con el número de acciones
    X = Dense(action_space, activation="linear", kernel_initializer='he_uniform')(X)

    model = Model(inputs=X_input, outputs=X, name='CartPole DQN model')
    model.compile(loss="mse", optimizer=RMSprop(lr=0.00025, rho=0.95, epsilon=0.01), metrics=["accuracy"])

    model.summary()
    return model

In [ ]:
class SARSA_Agent:
    def __init__(self):
        self.env = gym.make('CartPole-v1')
        # by default, CartPole-v1 has max episode steps = 500
        self.state_size = self.env.observation_space.shape[0]
        self.action_size = self.env.action_space.n
        self.EPISODES = 1000
        self.memory = deque(maxlen=2000)
        
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.001
        self.epsilon_decay = 0.999
        self.batch_size = 64
        self.train_start = 1000

        # create main model
        self.model = OurModel(input_shape=(self.state_size,), action_space = self.action_size)

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        if len(self.memory) > self.train_start:
            if self.epsilon > self.epsilon_min:
                self.epsilon *= self.epsilon_decay

    def act(self, state):
        if np.random.random() <= self.epsilon:
            return random.randrange(self.action_size)
        else:
            return np.argmax(self.model.predict(state))

    def replay(self):
        if len(self.memory) < self.train_start:
            return
        # Randomly sample minibatch from the memory
        minibatch = random.sample(self.memory, min(len(self.memory), self.batch_size))

        state = np.zeros((self.batch_size, self.state_size))
        next_state = np.zeros((self.batch_size, self.state_size))
        action, reward, done = [], [], []

        # do this before prediction
        # for speedup, this could be done on the tensor level
        # but easier to understand using a loop
        for i in range(self.batch_size):
            state[i] = minibatch[i][0]
            action.append(minibatch[i][1])
            reward.append(minibatch[i][2])
            next_state[i] = minibatch[i][3]
            done.append(minibatch[i][4])

        # do batch prediction to save speed
        target = self.model.predict(state)
        target_next = self.model.predict(next_state)

        for i in range(self.batch_size):
            # correction on the Q value for the action used
            if done[i]:
                target[i][action[i]] = reward[i]
            else:
                # Standard - DQN
                # DQN chooses the max Q value among next actions
                # selection and evaluation of action is on the target Q Network
                # Q_max = max_a' Q_target(s', a')
                target[i][action[i]] = reward[i] + self.gamma * (np.amax(target_next[i]))

        # Train the Neural Network with batches
        self.model.fit(state, target, batch_size=self.batch_size, verbose=0)


    def load(self, name):
        self.model = load_model(name)

    def save(self, name):
        self.model.save(name)
            
    def run(self):
        for e in range(self.EPISODES):
            state = self.env.reset()
            state = np.reshape(state, [1, self.state_size])
            done = False
            i = 0
            while not done:
                self.env.render()
                action = self.act(state)
                next_state, reward, done, _ = self.env.step(action)
                next_state = np.reshape(next_state, [1, self.state_size])
                if not done or i == self.env._max_episode_steps-1:
                    reward = reward
                else:
                    reward = -100
                self.remember(state, action, reward, next_state, done)
                state = next_state
                i += 1
                if done:                   
                    print("episode: {}/{}, score: {}, e: {:.2}".format(e, self.EPISODES, i, self.epsilon))
                    if i == 500:
                        print("Saving trained model as cartpole-dqn.h5")
                        self.save("cartpole-dqn.h5")
                        return
                self.replay()

    def test(self):
        self.load("cartpole-dqn.h5")
        for e in range(self.EPISODES):
            state = self.env.reset()
            state = np.reshape(state, [1, self.state_size])
            done = False
            i = 0
            while not done:
                self.env.render()
                action = np.argmax(self.model.predict(state))
                next_state, reward, done, _ = self.env.step(action)
                state = np.reshape(next_state, [1, self.state_size])
                i += 1
                if done:
                    print("episode: {}/{}, score: {}".format(e, self.EPISODES, i))
                    break

if __name__ == "__main__":
    agent = SARSA_Agent()
    agent.run()
    #agent.test()

ValueError: Argument(s) not recognized: {'lr': 0.00025}

In [13]:
class DQN_Agent:
    def __init__(self):
        self.env = gym.make('CartPole-v1')
        # by default, CartPole-v1 has max episode steps = 500
        self.state_size = self.env.observation_space.shape[0]
        self.action_size = self.env.action_space.n
        self.EPISODES = 1000
        self.memory = deque(maxlen=2000)
        
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.001
        self.epsilon_decay = 0.999
        self.batch_size = 64
        self.train_start = 1000

        # create main model
        self.model = OurModel(input_shape=(self.state_size,), action_space = self.action_size)

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        if len(self.memory) > self.train_start:
            if self.epsilon > self.epsilon_min:
                self.epsilon *= self.epsilon_decay

    def act(self, state):
        if np.random.random() <= self.epsilon:
            return random.randrange(self.action_size)
        else:
            return np.argmax(self.model.predict(state))

    def replay(self):
        if len(self.memory) < self.train_start:
            return
        # Randomly sample minibatch from the memory
        minibatch = random.sample(self.memory, min(len(self.memory), self.batch_size))

        state = np.zeros((self.batch_size, self.state_size))
        next_state = np.zeros((self.batch_size, self.state_size))
        action, reward, done = [], [], []

        # do this before prediction
        # for speedup, this could be done on the tensor level
        # but easier to understand using a loop
        for i in range(self.batch_size):
            state[i] = minibatch[i][0]
            action.append(minibatch[i][1])
            reward.append(minibatch[i][2])
            next_state[i] = minibatch[i][3]
            done.append(minibatch[i][4])

        # do batch prediction to save speed
        target = self.model.predict(state)
        target_next = self.model.predict(next_state)

        for i in range(self.batch_size):
            # correction on the Q value for the action used
            if done[i]:
                target[i][action[i]] = reward[i]
            else:
                # Standard - DQN
                # DQN chooses the max Q value among next actions
                # selection and evaluation of action is on the target Q Network
                # Q_max = max_a' Q_target(s', a')
                target[i][action[i]] = reward[i] + self.gamma * (np.amax(target_next[i]))

        # Train the Neural Network with batches
        self.model.fit(state, target, batch_size=self.batch_size, verbose=0)


    def load(self, name):
        self.model = load_model(name)

    def save(self, name):
        self.model.save(name)
            
    def run(self):
        for e in range(self.EPISODES):
            state = self.env.reset()
            state = np.reshape(state, [1, self.state_size])
            done = False
            i = 0
            while not done:
                self.env.render()
                action = self.act(state)
                next_state, reward, done, truncated, _ = self.env.step(action)
                next_state = np.reshape(next_state, [1, self.state_size])
                if not done or i == self.env._max_episode_steps-1:
                    reward = reward
                else:
                    reward = -100
                self.remember(state, action, reward, next_state, done)
                state = next_state
                i += 1
                if done:                   
                    print("episode: {}/{}, score: {}, e: {:.2}".format(e, self.EPISODES, i, self.epsilon))
                    if i == 500:
                        print("Saving trained model as cartpole-dqn.h5")
                        self.save("cartpole-dqn.h5")
                        return
                self.replay()

    def test(self):
        self.load("cartpole-dqn.h5")
        for e in range(self.EPISODES):
            state = self.env.reset()
            state = np.reshape(state, [1, self.state_size])
            done = False
            i = 0
            while not done:
                self.env.render()
                action = np.argmax(self.model.predict(state))
                next_state, reward, done, truncated, _ = self.env.step(action)
                state = np.reshape(next_state, [1, self.state_size])
                i += 1
                if done:
                    print("episode: {}/{}, score: {}".format(e, self.EPISODES, i))
                    break

if __name__ == "__main__":
    agent = SARSA_Agent()
    agent.run()
    #agent.test()

Model: "CartPole DQN model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 4)]               0         
                                                                 
 dense (Dense)               (None, 4)                 20        
                                                                 
 dense_1 (Dense)             (None, 16)                80        
                                                                 
 dense_2 (Dense)             (None, 2)                 34        
                                                                 
Total params: 134
Trainable params: 134
Non-trainable params: 0
_________________________________________________________________


c:\Users\danie\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

### ***Punto 3***
**Analice y compare los resultados obtenidos en las dos aproximaciones y escriba sus conclusiones.**